In [1]:
import sys
import os

# Ruta relativa desde el notebook a la carpeta de scripts
sys.path.append(os.path.abspath("../scripts"))

In [2]:
import numpy as np
import pandas as pd
import torch as t
from datasets import Dataset
from dataset_utils import MiSonGynyDataset, split_songs_into_verses
import task1_config as config

c:\Users\sdami\OneDrive\Escritorio\Misongyny\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset Creation

In [3]:
df = pd.read_csv("../datasets/subtask1_train.csv")

In [4]:
df.columns

Index(['id', 'lyrics', 'label'], dtype='object')

In [5]:
# T1_TRAIN_1918
# LiveGet tickets as low as $39

df[ df['id'] == 'T1_TRAIN_0616']

,id,lyrics,label
615,T1_TRAIN_0616,"[Letra de ""La Trayectoria (Mas Flow Family)"" f...",M


In [6]:
print(df['lyrics'][df['id'] == 'T1_TRAIN_0616'].to_numpy().tolist()[0])

[Letra de "La Trayectoria (Mas Flow Family)" ft. Héctor "El Father", Don Omar, Joan & O'Neill, Wibal & Alex]

Parte 1: Mas Flow Family (Don Omar, Joan & O'Neill, Wibal & Alex)

[Intro: Don Omar]
Probando, probando
Sube, sube Naldo
Coyote, te habla Luny
Tú sabe'; ¡Mas Flow Family!

[Coro 1: O'Neill]
Mas Flow Family
No va a haber quién nos pare, te digo
Mas Flow Family
De La Trayectoria, pa'l mundo entero (O'Neill)
Mas Flow Family
No va a haber quién nos pare, te digo
Mas Flow Family
De La Trayectoria, pa'l mundo entero

[Verso 1: O'Neill, Don Omar]
Soy yo, el que los coge y los parte en do'
Los míos siempre activa'o (Jaja), wherever they go
Who got flow? I'm the shit, I got flow
You know? Representó el corillo 'e Mas Flow (Coyote)

[Puente 1: Joan]
Los que se viren contra mí (¡Joan!)
Mi advertencia le' doy
Mi corillo esta activa'o (¡Oh-oh!)
Los que se viren contra mí (Contra mí)
Mi advertencia le' doy (Jajajaja)
Mi corillo esta activa'o (¡Oh-oh!)

[Verso 2: O'Neill]
Yo los ataco, los sa

In [7]:
songs  = split_songs_into_verses(df['lyrics'].to_numpy(), num_verses=10)
labels = df['label'].apply(lambda x: 0 if x == 'NM' else 1).to_numpy()
ids    = df['id'].to_numpy()

In [8]:
max( [len(verses) for verses in songs] ), min( [len(verses) for verses in songs] )

(10, 1)

In [11]:
n_verses = len([ (id, verses) for id, verses in zip(ids, songs) if len(verses) < 10 ])
print(f"number of verses with length less than 10: {n_verses}/{len(songs)}")

number of verses with length less than 10: 1529/2104


In [13]:
dataset = Dataset.from_dict({"songs": songs, "ids": ids, "labels": labels})

In [14]:
dataset.save_to_disk('../datasets/task1_mil_v2')

Saving the dataset (1/1 shards): 100%|██████████| 2104/2104 [00:00<00:00, 167590.55 examples/s]
